Welcome to week 8 part 1 of the [Noisebridge Python Class](https://github.com/audiodude/PythonClass)!

This class will be a little different. It covers the basics of web app development, using the excellent Flask package. However, Flask can't run inside a Jupyter notebook. Therefore, the different sets of code for this class are inside of the different folders in this directory. The code must be run using a local Python installation and the command line, which we will cover briefly.

[Here](https://python.land/installing-python) are some instructions for installing Python on Windows, macOS, or Linux.

In this lesson, you will learn:

* Setting up a Python development environment with virtualenv
* A basic "hello world" Flask app and how to run it
* Context on old school (CGI) web apps
* Using HTML templates with your Flask app, including template inheritance
* URL params in Flask
* GET params
* POST params, and when to use each

Let's get started!

First, we must set up a [virtualenv](https://docs.python.org/3/library/venv.html) for our code. A virtualenv allows us to install certain Python libraries/packages in an isolated environment. Suppose you're working on multiple Python projects on your computer. You don't want the libraries from one app to be included in a different app, especially if they have different required versions.

We can create a virtualenv very simply in Python 3. First we create a directory for our app (this has already been done in this repo):

```
mkdir intro-flask
```

Then we switch into that directory and create a virtualenv (this is required, because the virtualenv is never checked into source control directly):

```
cd intro-flask
python -m venv venv
```

In case you're wondering, the `-m venv` argument tells python to run the `venv` module, and the first and only argument to that module is also the string `venv`.

Then we can activate our virtualenv:

```
source venv/bin/activate
```

We must repeat this command in every new shell/command/terminal window once we are inside the `intro-flask` directory, to make sure we are using our virtualenv. You might notice that your shell prompt now starts with `(venv)` to tell you that you're using the virtualenv. If you need to stop using it for any reason (maybe to start using a different one) you can use simply use the command:

```
deactivate
```

Once we've created and activated our virtualenv, we can install the `flask` package using `pip`, the Python package manager. The `pip` command will automatically install any additional packages that are needed for the package you requested:

```bash
pip install flask
```

You should see a bunch of text fly by as Python downloads and installs the necessary package(s).

Finally, it is common to save a snapshot of the packages you are using, so that others who wish to use your code, or perhaps yourself in the future, can easily re-install the exact versions that were known to be working with your code. To do this, we use `pip freeze`. First, run `pip freeze` on its own, to see the output:

```
$ pip freeze
blinker==1.6.2
click==8.1.6
Flask==2.3.2
importlib-metadata==6.8.0
itsdangerous==2.1.2
Jinja2==3.1.2
MarkupSafe==2.1.3
Werkzeug==2.3.6
zipp==3.16.2
```

Then you can redirect the output of the command to a file (a common command line/bash operation). We will call this file `requirements.txt` as is common in the Python world.

```
pip freeze > requirements.txt
cat requirements.txt
```

When we run `cat` on requirements.txt, we see it's content, which should be the same as when we ran `pip freeze`.

When you need to install your requirements again, use the `-r` flag to pip install:

```
pip install -r requirements.txt
```

---

It is finally time to write our first "hello world" web app. In Flask, this is as simple as the following code, which you can find in `intro-flask/01-hello-world/main.py`. *Note that this code will do nothing in the Jupyter notebook.*

In [1]:
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
    return 'Hello World!'

If you recall from the lesson on web scraping, web sites run from applications known as **servers**. The server is always running, and waits for a **request** from a client, usually a web browser. When the server receives a request, it responds with whatever it's been programmed or configured to send back to the client. In this example, the server will always send back the plain text `Hello World!`.

The code above imports the flask library, then creates a new Flask app using `__name__` as its name, where `__name__` is a special Python variable which represents the name of the currently running module.

Next, the code uses the **decorator** `@app.route('/')` to modify the function right below it (`def index():`). This decorator tells Flask that the `index` function should be called when a request is made for the server path `/`, which is the root of the server and the path clients request when they visit the website without any path specified. So, for example, when you request `https://en.wikipedia.org` in your web browser, you are actually requesting the path `/` on the server `en.wikipedia.org`.

Let's start our server. First change into the directory containing the above code:

```
cd intro-flask/01-hello-world
```

Then, we use the `flask` command to start an HTTP server with our hello world Flask app:

```
FLASK_APP=main flask run
```

The portion `FLASK_APP=main` sets an [environment variable](https://www.digitalocean.com/community/tutorials/how-to-read-and-set-environmental-and-shell-variables-on-linux) which is a variable that is availble to all code running in the current shell. Since we specified it immediately before our command, it is only set and active for that one command.

The portion `flask run` starts the `flask` command with the argument `run`. The `flask` command is available in your environment via pip and the virtualenv you installed. If you get an error like `flask: command not found`, it is likely that your virtualenv is not set up correctly or you haven't installed flask. 

If your `flask` command runs successfully, you should see the following:

```
$ FLASK_APP=main flask run
 * Serving Flask app 'main'
 * Debug mode: off
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
```

Now let's open a web browser and check out our new web app! Open a new browser window and use either the URL that the Flask printed out (`http://127.0.0.1:5000`) or the equivalent shortcut `http://localhost:5000`. Web app developers generally use `localhost` because it is shorter and easier to type, but it is equivalent to `127.0.0.1`.

You should see:

![Screenshot of web app display 'Hello World!' in a browser window](https://pxscdn.com/public/m/_v2/588554065884192073/c51ea9690-856390/F9rl3ISqzW8l/BgUbbihwvrj2QakNzCUjOF75ODDFv54iJRaPWayl.png)

The text in the browser is the text that we setup in our `index` function.

You may also have noticed some print out in your terminal:

```
127.0.0.1 - - [06/Aug/2023 09:42:03] "GET / HTTP/1.1" 200
```

This a **log** of the request that was made by your browser, and indicates that a successful response of 200 was sent back (recall from a previous lesson that 200 represents success in HTTP).

---

The concept of a "web app" is not new. Since the mid-90s, programmers have used things such as [CGI](https://en.wikipedia.org/wiki/Common_Gateway_Interface) scripts written in the [Perl](https://en.wikipedia.org/wiki/Perl) language to create dynamic websites. Many oif these applications responded to user input and stored data. CGI (Common Gateway Interface) is a standard specifying how the web server ([Apache](https://en.wikipedia.org/wiki/Apache_HTTP_Server), [nginx](https://en.wikipedia.org/wiki/Nginx)) can pass necessary variables to the script, while reading the output of the script and sending the data back to the client.

It may be important to recognize how much simpler the Python/Flask approach to this problem is. Python uses a standard called [WSGI](https://www.toptal.com/python/pythons-wsgi-server-application-interface) (Web Server Gateway Interface) that allows it to separate the concerns of serving web pages from serving dynamic script content. Notice that we don't have to do things like parse the incoming file path directly and route our request accordingly, we just include the `@app.route` decorator. Additionally, common HTTP headers and status codes are handled automatically and we can focus on the parts of our application that really matter, such as rendering the output content.

---

Recall that the "source code" of the web is HTML. HTML is represented as **documents** which are served to the browser. In some cases, you can have a **static** HTML document, that exists as a file on the server. So when you go to `https://www.nasa.gov/kidsclub/index.html`, the server looks for a file called `index.html` in the `kidsclub` directory and serves it directly to the browser.

However, for web apps, we create HTML documents **dynamically**. The document doesn't exist as a file. In fact, it doesn't exist at all until the browser request is handled by the server. Then, the server constructs the document and returns it to the browser. This is how, for example, all Wordpress sites operate (except that the programming language used is [PHP](https://www.php.net/) not Python).

For our "hello world" example, we didn't return an HTML document. We returned plain text, which the browser interpreted and displayed appropriately. The basic skeleton of an HTML document is the following:

```
<!DOCTYPE html>
<html>
<head>
  <title>Hello world website</title>
</head>
<body>
  Hello World!
</body>
</html>
```

The `<!DOCTYPE` declaration informs the browser which version of HTML we are using. In modern times, we always use `<!DOCTYPE html>` to indicate the latest version of HTML.

The `<body>` tag is where we put the content that will be displayed in the browser.

What if we didn't want to always say hello, but also wanted to print the current time?

In [2]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
    return 'Hello World! ' + str(datetime.datetime.now())

![Screenshot of hello world application with time displayed](https://pxscdn.com/public/m/_v2/588554065884192073/c51ea9690-856390/sXxD1TtoRkVL/C1owtRyIAo6snhgptYaJh8zPzHmfzCRKnyPebJPk.png)

Now our hello world web app is truly dynamic, it changes every time the page is refreshed!

Imagine though, that we want to serve an HTML version of this data. We could use string formatting to achieve the result.

In [3]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Hello World! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

*(See this example in 02-hello-datetime)*

But what if we also wanted to specify a "Goodbye" page? The most obvious way is the following (where we use a `@app.route('/goodbye')` decorator to indicate that the Goodbye should be shown when the url `/goodbye` is requested):

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Hello World! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

@app.route('/goodbye')
def goodbye():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Goodbye! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

![Screenshot of web app showing Goodbye and the time](https://pxscdn.com/public/m/_v2/588554065884192073/c51ea9690-856390/Nk5YduPx9ZBX/O10nIHwVaTa3UkGsneBEvg1DlTKLx04n6Do14cY8.png)

We see now that we have a lot of duplicate code! Maybe we can refactor it a bit:

In [4]:
import datetime

import flask

app = flask.Flask(__name__)

def get_document(greeting, time_value):
  return f'''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      {greeting} {time_value}
    </body>
    </html>'''

@app.route('/')
def index():
  return get_document('Hello World!', datetime.datetime.now())

@app.route('/goodbye')
def goodbye():
  return get_document('Goodbye!', datetime.datetime.now())

*(See example in 03-inferior-templating)*

One thing to note about this approach is that `get_document` is just a normal Python function. We can define those alongside our web app or in separate modules that we import.

Using a helper function like this is the idea behind **templates** or **templating**. Most of the HTML document stays the same, but we have a few sections that we'd like to specify as variables. Luckily, Flask supports a powerful templating system, using [Jinja2](https://jinja.palletsprojects.com/en/3.1.x/). We can use Flask's templating system to keep our HTML templates in their own files, separate from our code, which makes them easier to edit and keeps our app logic cleaner.

First, we create a directory alongside our main module called `templates`:

```
mkdir templates
```

Then we create a file inside the templates directory, named `greeting.html`:

```
<!DOCTYPE html>
<html>
  <head>
    <title>Hello World website</title>
  </head>
  <body>
    {{ greeting }} {{ time_value }}
  </body>
</html>
```

The name of the template doesn't matter, we will refer to it directly in code.

Finally we update our application logic:

In [5]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return flask.render_template(
    'greeting.html',
    greeting='Hello World!',
    time_value=datetime.datetime.now())

@app.route('/goodbye')
def goodbye():
  return flask.render_template(
    'greeting.html',
    greeting='Goodbye!',
    time_value=datetime.datetime.now())

This is great, now our app logic is very simple and clean, and all of the HTML boilerplate that is repeated between the two documents is isolated in the template.

We can also capture parameters directly inside of our URLs using < and > delimiters. Here is an updated version of the code, as it is in `04-templating`:

In [ ]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/goodbye')
def goodbye():
  return flask.render_template(
    'greeting.html',
    greeting='Goodbye!',
    time_value=datetime.datetime.now())

@app.route('/greet')
def greet():
  return flask.render_template(
    'greeting.html',
    greeting='Hello World!',
    time_value=datetime.datetime.now())

@app.route('/greet/<name>')
def greet_by_name(name):
  greeting = f'Hello {name}!'
  return flask.render_template(
    'greeting.html',
    greeting=greeting,
    time_value='')

Notice that the `greet` *function* takes a `name` positional argument. This corresponds to the `<name>` portion of the route. So `http://localhost:5000/greet/Mateo` will return the text "Hello Mateo" (with no timestamp because `time_value` is empty string).

Another interesting property of this setup is that there's nothing being served on the root path (`/`). So if you go to `http://localhost:5000/` you get a 404 NOT FOUND error.

**Exercise:** Now try to write a new endpoint at http://localhost:5000/time/ that accepts a named URL parameter. The parameter should be the last part of the URL and should accept values of either `'year'` or `'month'`. If it's `'year'`, display the year value of the current time. If it's `'month'`, display the month. Remember to use <> in your route decorator so that you can capture the parameter, and specify the parameter name in the decorated function definition. You can reuse the `greeting.html` template, possibly with an empty `greeting`. Also, you can decide what to do if your parameter isn't 'year' or 'month'.

If you've forgotten, you can look at the [docs for `strftime`](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) 

---

What if we wanted to use a form to allow the user to specify the name that should be greeted? Granted, this is a very "Web 1.0" approach and generally wouldn't be used by most production applications today, but it definitely still works and is totally valid!

First we need a form that the user can fill out. Let's put it at the root path:

In [ ]:
@app.route('/')
def index():
  return flask.render_template('index.html')

```
    <form action="/greet" method="get">
      <label for="name">Greeting:</label>
      <input type="text" id="name" name="name" />
      <input type="submit" value="Submit" />
    </form>
```

WHen you fill out the form and hit the "Submit" button, the browser sees that the form has `action="/greet"` and it should use the `get` method, so it creates the URL `http://localhost:5000/greet?name=Mateo` and sends you there. The part `?name=Mateo` is known as the "GET parameters". You probably see these in URLs all day long. In our Python code, we will write simple code that will "extract" the values of these parameters and do something useful (like greet someone).

Now we have to write the function that processes this URL endpoint. Luckily, we can specify multiple routes in Flask that resolve to the same endpoint function.

*This code is in 05-get-params*

In [ ]:
@app.route('/greet')
@app.route('/greet/<name>')
def greet_by_name(name=None):
  # If name isn't None, it means that Flask passed it
  # to our function, because it was part of the URL.
  #
  # Otherwise we got here by GET params, not /greet/Foo,
  # so we get the params from the GET args. Like a dict,
  # the default value is used if the param isn't present.
  if name is None:
    name = flask.request.args.get('name', 'World')
    
  greeting = f'Hello {name}!'
  return flask.render_template(
    'greeting.html',
    greeting=greeting,
    time_value='')

So this function is run if either 1) The user typed `/greet/SomeName` into their browser (or was otherwise linked there), or 2) if the user pressed "Submit" on the index page.

GET parameters are used in the case where the web page displays static information about something based on a parameter. So like `https://baseball.example.com/team?name=Red+Sox`. Such a page would show a page about the Boston Red Sox. The page is still dynamic, it's likely pulling information from a database each time it is accessed, and if the data changes in that database it would display different information. It's convenient to use GET params in this case, because someone can send you the link `https://baseball.example.com/team?name=Red+Sox` and you can immmediately see what they saw. This isn't any kind of requirement though, that the page shows the same thing when the same GET params are sent.

The other way parameters can be passed to a web app endpoint is `POST` parameters. POST parameters are sent in the "body" of the HTTP request (requests can have body sections just like responses). The most obvious example of POST requests is login forms. You don't want someone to accidentally share a link like `https://login.example.com/?user=mikesmith&password=hotguy123`. Additionally, POST params are used more generally when the user is taking an "action". By default, web browsers won't re-POST these requests, so they prevent the action from happening multiple times. It would be really silly to have `https://radish.example.com/post?url=foo.com` and have it create a new entry every time that URL is requested, for instance.

---

These concepts will be discussed further and expanded on in Part 2 of this lesson, where we will build a fully functioning web app.